# **Viktig**: Kjør cellen nedenfor for å laste inn OpenAI API-nøkkelen for resten av notebooken.

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
from typing import Optional
_ = load_dotenv(find_dotenv()) # read local .env file
doc_address = 'resources/Noria_eBook_The_Insurance_Industry_2025_171205_v8.pdf' #PDF file address for question answering.

# Hva er Retrieval Augmented Generation (RAG)?

LLM-er har begrenset kunnskap og mangler tilgang til informasjon utenfor treningssettet. Dette begrenser deres evne til å presentere fersk og oppdatert informasjon uten en kontekst om spesifikke emner. I denne delen vil vi utforske måter å tilføre nødvendig kontekst eksternt, slik at LLM-ene kan dra nytte av sin resonnering for å svare på spørsmål basert på korrekte informasjonskilder.

Retrieval Augmented Generation (RAG) går ut på å øke språkmodellens (LLM) kunnskap ved å integrere ekstra data. Selv om LLM-er kan håndtere en rekke emner er deres forståelse begrenset til offentlig tilgjengelig informasjon opp til et spesifikt treningspunkt. For å muliggjøre resonnering om private data eller data etter treningsperioden er det nødvendig å berike modellens kunnskap med den spesifikke informasjonen som kreves. Denne prosessen, kjent som Retrieval Augmented Generation (RAG), innebærer å hente og inkludere relevant informasjon i modellens prompt.

LangChain omfatter ulike komponenter som er spesielt utviklet for å forenkle utviklingen av Q&A-applikasjoner og, mer generelt, RAG-applikasjoner.

## RAG består av to hoveddeler:
### 1. Retrieval og generering
#### 1. Retrieval: Ved en gitt brukerinput hentes relevante segmenter fra lagring ved hjelp av en Retriever.
#### 2. Generer: En ChatModel / LLM produserer en respons ved hjelp av et prompt som inkluderer spørsmålet og den innhentede informasjonen.

<div style="display: flex;  height: 500px;">
    <img src="resources/RAG.png"  style="margin-left:auto; margin-right:auto"/>
</div>

Vi skal gå gjennom hvert trinn og konstruere et RAG-system ved ved hjelp av LangChain slik at vi kan chatte med en PDF-fil.

# 1. Retrieval

I dette eksempelet tar vi en nærmere titt på retrieval fra en PDF-fil. Den samme tilnærmingen kan utvides til data i en database, et nettsted, osv.

Det første trinnet involverer preprosessering av PDF-filen og består vanligvis av tre steg:

1. **Load**: Begynner med å laste inn dataene ved hjelp av ```DocumentLoaders```.
2. **Split**: Tekstdelere bryter ned store ```Dokumenter``` i mindre deler. Dette er nyttig både for indeksering av data og for bruk i kombinasjon med en modell, ettersom store deler gir dårligere søkeresultater og ikke passer inn i modellens begrensede kontekstvindu.
3. **Store**: Vi trenger et sted å lagre splittet og indeksert data slik at den er tilgjengelig for søk. Dette gjøres oftest ved hjelp av en VectorStore og Embeddings-modell.

Vi skal nå gå gjennom hvert steg med tilhørende kode.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

Vi har tilgjengeliggjort en eksempel PDF. Dokumentet består av 25 sider med tekst og på grunn av lengden er det ikke mulig å mate direkte til en LLM. Derfor er det nødvendig å utføre splitting og indeksering.

In [ ]:
from IPython.display import display, IFrame

# Specify the path to your PDF file
pdf_path = doc_address

# Create an IFrame to embed the PDF viewer
pdf_viewer_iframe = IFrame(src=pdf_path, width=1100, height=900)

# Display the IFrame
pdf_viewer_iframe

## 1.1 Last inn filen

<div style="display: flex;  height: 500px;">
    <img src="resources/doc_load.png"  style="margin-left:auto; margin-right:auto"/>
</div>

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


# Decode the file
loader = PyPDFLoader(doc_address)

# Check out the text from the PDF
loader.load()[:3]

## 1.2 Splitt teksten

<div style="display: flex;  height: 500px;">
    <img src="resources/doc_split.png"  style="margin-left:auto; margin-right:auto"/>
</div>

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

pages = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)



# Split the document into chunks
splits = text_splitter.split_documents(pages)

splits[:3]

## 1.3 Embed og lagre teksten

<div style="display: flex;  height: 500px;">
    <img src="resources/doc_embed_store.png"  style="margin-left:auto; margin-right:auto"/>
</div>

In [ ]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


# Define the embedding function for the document text
embeddings = OpenAIEmbeddings()

# Create a Chroma vector for searching the documents
docsearch = Chroma.from_documents(splits,
                                 embeddings)

Et eksempel på en embedded vektor for en av splittene er vist i cellen nedenfor:

In [ ]:
docsearch.get([docsearch.get()['ids'][5]], include=['embeddings', 'documents'])

Du kan stille forskjellige spørsmål og de splittene med nærmeste kontekst vil bli kalt ved hjelp av ```Chroma``` API.

In [ ]:
docsearch.as_retriever().get_relevant_documents('What is Peer-to-Peer (P2P) Insurance?')

# 2. RAG Agent

<div style="display: flex;  height: 500px;">
    <img src="resources/RAG_agent.png"  style="margin-left:auto; margin-right:auto"/>
</div>

Vi skal nå bruke ```LangGraph```, som vi ble kjent med i forrige notebook, for å utvikle en langchain-agent som har tilgang til retrieval-motoren definert som et tool. For å oppnå dette må vi følge to trinn:

1. Definer retrieval-tool ved hjelp av innebygde ```langchain```-funksjoner.
2. Lag agent-pipline-grafen ved hjelp av ```LangGraph```.

Koden for trinnene er i følgende celle:

In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langgraph.prebuilt import ToolExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain_core.messages import BaseMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolInvocation
from langgraph.graph import END, StateGraph

prompt = PromptTemplate.from_template("Page {page}: {page_content}")
doc_sep = '========='

tool = create_retriever_tool(
    docsearch.as_retriever(),
    "retrieve_insurance_doc",
    "Use this tool to answer any question about the insurance and finance using the Noria documents",
    document_prompt= prompt,
    document_separator=doc_sep
)

tools = [tool]

# We will set streaming=True so that we can stream tokens
model = ChatOpenAI(temperature=0, streaming=True)

functions = [format_tool_to_openai_function(t) for t in tools]

model_with_functions = model.bind_functions(functions)

tool_executor = ToolExecutor(tools)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]



### Edges


def should_retrieve(state):
    """
    Decides whether the agent should retrieve more information or end the process.

    This function checks the last message in the state for a function call. If a function call is
    present, the process continues to retrieve information. Otherwise, it ends the process.

    Args:
        state (messages): The current state of the agent, including all messages.

    Returns:
        str: A decision to either "continue" the retrieval process or "end" it.
    """
    print("---DECIDE TO RETRIEVE---")
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        print("---DECISION: DO NOT RETRIEVE / DONE---")
        return "end"
    # Otherwise there is a function call, so we continue
    else:
        print("---DECISION: RETRIEVE---")
        return "continue"


### Nodes


# Define the function that calls the model
def call_model(state):
    """
    Invokes the agent model to generate a response based on the current state.

    This function calls the agent model to generate a response to the current conversation state.
    The response is added to the state's messages.

    Args:
        state (messages): The current state of the agent, including all messages.

    Returns:
        dict: The updated state with the new message added to the list of messages.
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    response = model_with_functions.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function to execute tools
def call_tool(state):
    """
    Executes a tool based on the last message's function call.

    This function is responsible for executing a tool invocation based on the function call
    specified in the last message. The result from the tool execution is added to the conversation
    state as a new message.

    Args:
        state (messages): The current state of the agent, including all messages.

    Returns:
        dict: The updated state with the new function message added to the list of messages.
    """
    print("---EXECUTE RETRIEVAL---")
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # print(type(response))
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", call_model)  # agent
workflow.add_node("action", call_tool)  # retrieval


# Call agent node to decide to retrieve or not
workflow.set_entry_point("agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    should_retrieve,
    {
        # Call tool node
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge('action', 'agent')

# Compile
agent = workflow.compile()

In [ ]:
import pprint

from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content="What is the future of insurance?"
        )
    ]
}

for output in agent.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")



## Prøv forskjellige spørsmål til dokumentet som er lastet inn.

In [ ]:
import pprint

from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content="..." ## write the question instead of three dots.
        )
    ]
}

for output in agent.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# TODO: Chatbot med RAG

Til slutt ønsker vi å lage et chatbot-grensesnitt for agenten. For å gjøre dette skal vi bruke ***[holoviz Panel library](https://panel.holoviz.org/index.html)*** og lage en klasse for chatbotten. Fyll ut TODO-delene av koden for å fullføre chatbotten og få svar på sprørsmålene til PDF-filen. Chatbot-klassen bruker samme agent som er definert ovenfor. 
#### !!Det er derfor viktig at alle celler ovenfor er kjørt for at agenten skal fungere

Fyll ut feltene angitt med #-kommentarer og fullfør chatbot-klassen og -funksjonene for å se chatboten i aksjon.

In [ ]:
import panel as pn
import fitz
from PIL import Image
from functools import partial
import param
import re
from langchain_core.messages.function import FunctionMessage
from langchain.schema.messages import AIMessage
from panel.chat import ChatMessage

pn.extension()

def chat_handler(contents, user, instance):
    # TODO: Try playing around with ChatMessage class to create fancy responses.
    # use the instance.generate_response(contents) function to return the answer to the user. 

    return instance.generate_response(contents)

class PDFChatBot(pn.chat.ChatInterface):
    """
    A HoloViz Panel extension providing a front end for a chatbot equipped with RAG tool.

    This class extends the `pn.chat.ChatInterface` widget to integrate with a chatbot interface
    implemented in Python. It provides a user-friendly chat interface within a Panel
    application, allowing users to interact with the underlying chatbot.

    Attributes:
    -----------
    callback: function
        The function to handle the response to the user chat message.
        
    callback_user: str
        The name of the chatbot user.

    

    Methods:
    --------
    switch_source_tab:
        Switches the active tab to show the source page.
    layout:
        Returns the Panel layout containing the chat interface and other components.
    generate_response:
        Generates a response based on the user query and chat history.
    render_page:
        Renders a specific page of a PDF file as an image.

    """
    def __init__(self, *objects, **params):
        """
        Initialize the PDFChatBot.
        """
        self.callback = chat_handler
        self.callback_user = 'Assistant'
        super(PDFChatBot, self).__init__( *objects, **params)



        ##############################################################
        ## The page numbers of the returned source material from retrieval engine
        ## an array to keep track of chat history
        ## the langchain agent defined above
        
        self.srouce_page_num = [0, 1, 2]
        self.chat_history = []
        self.agent = agent
        ##############################################################

        
        
        self.source_images = [pn.pane.Image(width=500) for _ in range(3)]
        

        self.source_pane = pn.layout.Row(*self.source_images)

        
        
        self.chage_source_tab_button = pn.widgets.Button(name="Show source", button_type='primary')
        # Connect the button click event to the method
        self.chage_source_tab_button.on_click(self.switch_source_tab)


        self.tabs = pn.Tabs(('Conversation', self), ('Show source page', self.source_pane))
        
        self.render_page()

    ##############################################################
    ### Compelete the below function definition.
    ## the aim of the function is to answer the input user query using self.agent.
    ## if the function call has happened, make the necessary adjustments to update the source pages.
    
    def generate_response(self, query):
        ##TODO: complete this function to generate the response to the user query, 
        ## Determine whether the openAI function has been called.
        """
        Generate a response based on user query and chat history.

        Parameters:
        -----------
        query : str
            User's query.

        Returns:
        --------
        answer : str
            Returned output from the agent.
        function_called : bool
            Indicates if a function was called in the response.
        """
        inputs = {
            "messages": [
                HumanMessage(
                    content=query
                )
            ]
        }

        # TODO: check for function calls and adjust the reponse of the model accordingly.
        # The langchain agent is in class property self.agent, you can call it with self.agent.invoke(inputs) 
        # If the retrieval function is called you need to update the source page numbers from the reponse.
        answer = 'default response'
        return answer


    

    def switch_source_tab(self, event):
        """
        Switches the active tab to show the source page.

        Parameters:
        -----------
        event : Event
            The event object representing the button click event.
        """
        self.tabs.active = 1 if self.tabs.active == 0 else 1

    def layout(self):
        """
        Returns the Panel layout containing the chat interface and other components.

        Returns:
        --------
        Panel: The Panel layout containing the chat interface and other components.
        """
        return self.tabs

    def render_page(self):
        """
        Renders source pages of a PDF file in the source tab.

        Returns:
        --------
        None
        """
        doc = fitz.open(doc_address)

        
        for i, pdf_page in enumerate(self.srouce_page_num[:3]):
            page = doc[pdf_page]
            pix = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
            image = Image.frombytes('RGB', [pix.width, pix.height], pix.samples)
            self.source_images[i].param.update(object=image)

    


In [ ]:
chatbot = PDFChatBot()

In [ ]:
chatbot.layout()

# Du kan prøve dette med andre pdf-filer. Last opp din egen pdf-fil og endre filbanen øverst i denne notebooken.